<a href="https://colab.research.google.com/github/zoraizmohammad/quantum-edupca/blob/main/pca/quantumPCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Quantum PCA**

In [ ]:
import numpy as np
import pandas as pd
from qiskit import QuantumCircuit,transpile, transpile
from qiskit_aer import Aer
from qiskit.circuit.library import QFT
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

**Necessary Installs:** qiskit, qiskit_aer

In [ ]:
!pip install qiskit
!pip install qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 82.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load Data

In [18]:
# Load the synthetic educational dataset
data = pd.read_csv("/content/drive/MyDrive/projects/synthetic_educational_data.csv")

Feature Selection

In [20]:
# Select numerical features for PCA
data_numeric = data.drop(columns=[col for col in data.columns if col.startswith('Style')])
data_matrix = data_numeric.to_numpy()

Calculate the Covariance Matrix

In [21]:
cov_matrix = np.cov(data_matrix.T)
print("Covariance Matrix:")
print(cov_matrix)

Covariance Matrix:
[[ 2.24967241e+02 -2.64747275e-01 -1.09748862e+01  6.58386511e-02
   2.56883396e-01]
 [-2.64747275e-01  3.11979495e-02 -1.22343728e-01  1.56826151e-03
  -1.60150731e-03]
 [-1.09748862e+01 -1.22343728e-01  2.71038492e+02 -2.68028646e-01
   7.69435932e-02]
 [ 6.58386511e-02  1.56826151e-03 -2.68028646e-01  2.04584463e-02
  -3.32094433e-03]
 [ 2.56883396e-01 -1.60150731e-03  7.69435932e-02 -3.32094433e-03
   4.44871211e-02]]


Normalize Covariance Matrix

In [22]:
norm_factor = np.linalg.norm(cov_matrix)
normalized_matrix = cov_matrix / norm_factor
print("Normalized Covariance Matrix:")
print(normalized_matrix)

Normalized Covariance Matrix:
[[ 6.38057994e-01 -7.50883170e-04 -3.11272602e-02  1.86733310e-04
   7.28579428e-04]
 [-7.50883170e-04  8.84844430e-05 -3.46994492e-04  4.44794446e-06
  -4.54223707e-06]
 [-3.11272602e-02 -3.46994492e-04  7.68726483e-01 -7.60189880e-04
   2.18229438e-04]
 [ 1.86733310e-04  4.44794446e-06 -7.60189880e-04  5.80247824e-05
  -9.41894945e-06]
 [ 7.28579428e-04 -4.54223707e-06  2.18229438e-04 -9.41894945e-06
   1.26175540e-04]]


Prepare Quantum States Function Definition

In [23]:
def prepare_quantum_state(matrix):
    flattened_matrix = matrix.flatten()
    norm = np.linalg.norm(flattened_matrix)
    if norm == 0:
        raise ValueError("Matrix cannot be normalized; norm is zero.")
    state_vector = flattened_matrix / norm
    return Statevector(state_vector)